In [1]:
import tensorflow as tf
import pandas as pd
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
from tokenizers import CharBPETokenizer
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast, GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import random
import time, math, copy
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchtext
from tokenizers import CharBPETokenizer
import tokenizers
import operator
from queue import PriorityQueue

# specify GPU
device = torch.device("cuda")

# Import from external file
X_train = pd.read_csv("output/x_train.csv")
X_test = pd.read_csv("output/x_test.csv")

# Set paths
train_path = 'output/x_train_nn.csv'
test_path = 'output/x_test_nn.csv'

# Save files
X_train[['question', 'prep_answer', 'cluster']].to_csv(train_path, index=False, header=False)
X_test[['question', 'prep_answer', 'cluster']].to_csv(test_path, index=False, header=False)

In [2]:
tokenizer = CharBPETokenizer(bert_normalizer=False)
# train the tokenizer model
tokenizer.train('output/x_train_nn.csv')
# saving the tokenized data in our specified folder
save_path = 'tokenized_data'
tokenizer.save_model(save_path)

['tokenized_data\\vocab.json', 'tokenized_data\\merges.txt']

In [3]:
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path, return_tensors="tf")
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [4]:
single_string = ''
for i in range(len(X_train)):
    single_string += 'Cluster' + str(X_train['cluster'].iloc[i]) + ' <s> ' + X_train['prep_answer'].iloc[i] + ' </s> '
string_tokenized = tokenizer.encode(single_string)

In [5]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [6]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy()
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [7]:
num_epoch = 1
history = model.fit(dataset, epochs=num_epoch)

d:\programdata\venv\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


InvalidArgumentError:  indices[5,8] = 16729 is not in [0, 16728)
	 [[node tfgp_t2lm_head_model/transformer/wte/Gather (defined at d:\programdata\venv\lib\site-packages\transformers\modeling_tf_utils.py:905) ]] [Op:__inference_train_function_20238]

Errors may have originated from an input operation.
Input Source operations connected to node tfgp_t2lm_head_model/transformer/wte/Gather:
 tfgp_t2lm_head_model/transformer/Reshape (defined at d:\programdata\venv\lib\site-packages\transformers\models\gpt2\modeling_tf_gpt2.py:301)

Function call stack:
train_function
